# Remoção de ruídos de documentos utilizando Auto Encoder Convolucional

### Imports necessários

In [1]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras.utils import plot_model

import numpy as np
import matplotlib.pyplot as plt

from skimage.io import imread
from skimage.transform import resize,rescale
from os import listdir

from IPython.display import Image

Using TensorFlow backend.


### Função para carregar e montar os dados em memória para realizar o treinamento

Como nossos dados são imagens salvas no disco, precisamos de uma função customizada

In [2]:
def build_data():
    
    x_train = []
    
    for file in listdir('kdd/train'):
    
        img = imread('kdd/train/' + file)
        
        img = resize(img,(260,540),True)
        
        img = img.reshape(1,img.shape[0],img.shape[1])
    
        
        x_train.append(img)
        
    
    x_train = tuple(x_train)
    
    x_train = np.concatenate(x_train)
    
    y_train = [] 
    
    
    for file in listdir('kdd/train_cleaned'):
    
        img = imread('kdd/train_cleaned/' + file)
        
        img = resize(img,(260,540),True)
        
        y_train.append(img.reshape(1,img.shape[0],img.shape[1]))
    
    
    
    y_train = tuple(y_train)
    
    y_train = np.concatenate(y_train)
    
    x_test = []
    
    for file in listdir('kdd/test'):
    
        img = imread('kdd/test/' + file)
        
        img = resize(img,(260,540),True)
        
        x_test.append(img.reshape(1,img.shape[0],img.shape[1]))
    
    
    
    x_test = tuple(x_test)
    
    x_test = np.concatenate(x_test)
    
    return x_train,y_train,x_test
    


In [3]:
x_train, y_train, x_test = build_data()

/home/374184/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/374184/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


### Montando o tensor

O nosso autoencoder receberá um Tensor4D de arquitetura NHWC, contendo as imagens, como input

In [4]:
x_train = np.reshape(x_train, (len(x_train), 260, 540, 1))  # adapt this if using `channels_first` image data format
y_train = np.reshape(y_train, (len(x_train), 260, 540, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 260, 540, 1))  # adapt this if using `channels_first` image data format

### Divisão dos dados

Precisamos dividir os dados para termos treino validação e teste

In [5]:
x_validation = x_train[121:145]
x_train = x_train[0:120]

y_validation = y_train[121:145]
y_train = y_train[0:120]

In [6]:
input_img = Input(shape=(260,540,1))

downConv = Conv2D(32,(3,3),activation='relu',padding='same')(input_img)
downConv = MaxPooling2D((2,2),padding='same')(downConv)
downConv = Conv2D(32,(3,3),activation='relu',padding='same')(downConv)

encoded = MaxPooling2D((2,2),padding='same',name='encoded')(downConv)

encoder = Model(input_img,encoded,name='encoder')

plot_model(encoder, to_file='cae_encoder.png',show_shapes=True)

Image(retina=False, filename='cae_encoder.png',width=535,height=516)

Instructions for updating:
Colocations handled automatically by placer.


ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [ ]:
encoded_inputs = Input(shape=(65,135,32), name='encoded_inputs')
upConv = Conv2D(32,(3,3),activation='relu',padding='same')(encoded_inputs)
upConv = UpSampling2D((2,2))(upConv)
upConv = Conv2D(32,(3,3),activation='relu',padding='same')(upConv)
upConv = UpSampling2D((2,2))(upConv)


decoded = Conv2D(1,(3,3), activation='sigmoid', padding='same',name='decoded')(upConv)

decoder = Model(encoded_inputs,decoded,name='decoder')

plot_model(decoder, to_file='cae_decoder.png',show_shapes=True)

Image(retina=False, filename='cae_decoder.png',width=535,height=516)

In [ ]:
outputs = decoder(encoder(input_img))

autoencoder = Model(input_img,outputs,name='autoencoder')

plot_model(autoencoder, to_file='cae.png',show_shapes=True)

Image(retina=False, filename='cae.png',width=535,height=516)

In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

history = autoencoder.fit(x_train, y_train,
                epochs=100,
                batch_size=12,
                shuffle=True,
                validation_data=(x_validation, y_validation))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
denoised_imgs = autoencoder.predict(x_test)

n = 5
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(260, 540))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n + 1)
    plt.imshow(denoised_imgs[i].reshape(260, 540))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
#plt.show()
plt.savefig('kdd_result.png')

In [ ]:
plt.imshow(denoised_imgs[0].reshape(260, 540))

In [ ]:
plt.imsave('test.png', denoised_imgs[0].reshape(260, 540))